In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

import numpy as np
tf.keras.backend.set_floatx('float64')

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
print(tf.newaxis)

None


In [4]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float64")
x_test = x_test[..., tf.newaxis].astype("float64")

print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [5]:
# 10000 reshuffles iterations
train_ds = tf.data.Dataset.from_tensor_slices((x_train[:100], y_train[:100])).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test[:100], y_test[:100])).batch(32)

In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten(input_shape=(28, 28))
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x): # like forward in pytorch
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [8]:
# metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
# modify train and test steps
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [10]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 2.04587984085083, Accuracy: 41.0, Test Loss: 1.5746632516384125, Test Accuracy: 56.00000000000001
Epoch 2, Loss: 0.7676442861557007, Accuracy: 72.0, Test Loss: 1.1599482297897339, Test Accuracy: 67.0
Epoch 3, Loss: 0.33470661751925945, Accuracy: 91.0, Test Loss: 1.0206339657306671, Test Accuracy: 72.0
Epoch 4, Loss: 0.17306457925587893, Accuracy: 94.0, Test Loss: 1.0780822485685349, Test Accuracy: 66.0
Epoch 5, Loss: 0.18344879895448685, Accuracy: 96.0, Test Loss: 1.1276864856481552, Test Accuracy: 69.0
